## How to add metadata to your documents and filter searches
This notebook will walk you through how to upload metadata that provides extra information about the corpus you are ingesting with nv-ingest. It will show the requirements for the metadata file and what file types are supported. Then we will go throught he process of filtering searches, in this case, on the metadata we provided.

First step is to provide imports for all the tools we will be using.

In [1]:
from nv_ingest_client.client import Ingestor
from nv_ingest_client.util.milvus import nvingest_retrieval
import pandas as pd

/opt/conda/envs/nv_ingest_runtime/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Next we will annotate all the necessary variables to ensure our client connects to our pipeline.

In [2]:
model_name="nvidia/llama-3.2-nv-embedqa-1b-v2"
hostname="localhost"
collection_name = "nv_ingest_collection"
sparse = True

Now, we will begin by creating a dataframe with dummy metadata in it. The metadata can be ingested as either a dataframe or a file. Supported file types (json, csv, parquet). If you supply a file it will be converted into a pandas dataframe for you. In this example, after we create the dataframe, we write it to a file and we will use that file as part of the ingestion.

In [7]:
meta_df = pd.DataFrame(
    {
        "source": ["data/woods_frost.pdf", "data/multimodal_test.pdf"],
        "meta_a": ["alpha", "bravo"]
    }
)
file_path = "./meta_df.csv"
meta_df.to_csv(file_path)

If you are supplying metadata during ingestion you are required to supply three keyword arguments.

- meta_dataframe - This is either a string representing the file (to be loaded via pandas) or the already loaded dataframe.
- meta_source_field - This is a string, that represents the field that will be used to connect to the document during ingestion.
- meta_fields - This is a list of strings, representing the columns of data from the dataframe that will be used as metadata for the corresponding documents.

All three of the parameters are required to enable metadata updates to the documents during ingestion.


In [8]:
ingestor = ( 
    Ingestor(message_client_hostname=hostname)
    .files(["data/woods_frost.pdf", "data/multimodal_test.pdf"])
    .extract(
        extract_text=True,
        extract_tables=True,
        extract_charts=True,
        extract_images=True,
        text_depth="page"
    ).embed(text=True, tables=True
    ).vdb_upload(collection_name=collection_name, milvus_uri=f"http://{hostname}:19530", sparse=sparse, minio_endpoint=f"{hostname}:9000", dense_dim=2048
                 ,meta_dataframe=file_path, meta_source_field="source", meta_fields=["meta_a"]
                )
)
results = ingestor.ingest_async().result()

'text' parameter is deprecated and will be ignored. Future versions will remove this argument.
'tables' parameter is deprecated and will be ignored. Future versions will remove this argument.


Once the ingestion is complete, the documents will have uploaded to the vector database with the corresponding metadata as part of the `content_metadata` field. This is a json field that can be used as part of a filtered search. To use this, you can select a column from the meta_fields previously described and filter based on a value for that sub-field. That is what is done in this example below. There are more extensive filters that can be applied, please refer to https://milvus.io/docs/use-json-fields.md#Query-with-filter-expressions for more information.

In [9]:
queries = ["this is expensive"]
top_k = 5
q_results = []
for que in queries:
    q_results.append(nvingest_retrieval([que], collection_name=collection_name, host=f"http://{hostname}:19530", embedding_endpoint=f"http://{hostname}:8012/v1",  hybrid=sparse, top_k=top_k, model_name=model_name, gpu_search=False
                                            , _filter='content_metadata["meta_a"] == "alpha"'
                                           ))

print(f"{q_results}")

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /opt/conda/envs/nv_ingest_runtime/lib/python3.12/site-
[nltk_data]     packages/llama_index/core/_static/nltk_cache...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


[data: [[{'id': 458710425561758215, 'distance': 0.016393441706895828, 'entity': {'source': {'source_name': '/raid/nv-ingest/data/woods_frost.pdf', 'source_id': '/raid/nv-ingest/data/woods_frost.pdf', 'source_location': '', 'source_type': 'PDF', 'collection_id': '', 'date_created': '2024-04-30T18:02:30', 'last_modified': '2024-04-30T18:02:32', 'summary': '', 'partition_id': -1, 'access_level': -1}, 'content_metadata': {'type': 'text', 'description': 'Unstructured text from PDF document.', 'page_number': 0, 'hierarchy': {'page_count': 2, 'page': 0, 'block': -1, 'line': -1, 'span': -1, 'nearby_objects': {'text': {'content': [], 'bbox': [], 'type': []}, 'images': {'content': [], 'bbox': [], 'type': []}, 'structured': {'content': [], 'bbox': [], 'type': []}}}, 'subtype': '', 'start_time': -1, 'end_time': -1, 'location': None, 'max_dimensions': None, 'meta_a': 'alpha'}, 'text': 'Stopping by Woods on a Snowy Evening, By Robert Frost\r\nFigure 1: Snowy Woods\r\nWhose woods these are I think I 